In [1]:
from google.colab import drive
path = '/content/drive'
drive.mount(path)
folder_name = 'project1'
assert folder_name is not None, "[!] Enter the foldername."
import sys
sys.path.append('/content/drive/My Drive/{}'.format(folder_name))
%cd /content/drive/My\ Drive/$folder_name
%cd /content/drive/My\ Drive/$folder_name

Mounted at /content/drive
/content/drive/My Drive/project1
/content/drive/My Drive/project1


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


Not connected to a GPU


In [ ]:
!ls

Data		   helpers.py		   project1_description.pdf
data_utils.py	   implementations.py	   __pycache__
data_utils_yao.py  implementations_yao.py  Untitled0.ipynb
feedback	   latex-example-paper
grading_tests	   P1.ipynb


In [2]:
# Import basic libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_utils import *
from implementations import *

In [ ]:
!pwd

/content/drive/My Drive/project1


In [3]:
# Define some paths

data = './Data/'
train_set_root = os.path.join(data, 'train.csv')
test_set_root = os.path.join(data, 'test.csv')

In [4]:
train_data, train_label = load_data(train_set_root)


In [ ]:
train_data[: ,22]

array([2., 1., 1., ..., 1., 0., 0.])

In [ ]:
train_label

array([ 1., -1., -1., ...,  1., -1., -1.])

In [5]:
def compute_statistics_all(y, tx, w, func_type='linear'):
    """Compute Loss and Accuracy after training.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy arrays of shape (2, ), final weight
        func_type: type of model used, linear for linear regression and logistic for logistic regression.

    Returns:
        loss: scalar
        acc: classification accuracy
    """
    data_num = y.shape[0]

    if (func_type == 'linear'):
        model_output = np.dot(tx, w)
        e_mat = y - model_output
        loss = 0.5 * np.dot(e_mat.T, e_mat) / data_num
        model_output[np.where(model_output > 0)] = 1
        model_output[np.where(model_output <= 0)] = -1
    elif (func_type == 'logistic'):
        model_output = sigmoid(np.dot(tx, w))
        loss = -np.dot(y, np.log(model_output + eps)) - np.dot((1 - y).T, np.log(1 - model_output + eps))
        model_output[np.where(model_output > 0.5)] = 1
        model_output[np.where(model_output <= 0.5)] = 0
    
    acc = np.sum(model_output == y) / data_num
    return loss, acc

In [6]:
import data_utils
def cross_validation(train_data, train_label, k_fold, eval_func, input_dict):
  data_num = train_data.shape[0]
  data_dim = train_data.shape[1]
  fold_part_idx = data_utils.k_fold(data_num, k_fold)

  loss_list = []
  acc_list = []
  for k in range(k_fold):
    cur_train_idx = np.concatenate([fold_part_idx[:k, :], fold_part_idx[k+1:, :]], axis = 0)
    cur_train_idx = cur_train_idx.flatten()
    cur_test_idx = fold_part_idx[k, :]
    x_tr = train_data[cur_train_idx, :]
    x_te = train_data[cur_test_idx, :]
    y_tr = train_label[cur_train_idx]
    # print(np.sum(y_tr==1)/len(y_tr))
    y_te = train_label[cur_test_idx]

    lambda_ = input_dict.get('lambda_')
    initial_w = input_dict.get('initial_w')
    max_iters = input_dict.get('max_iters')
    gamma = input_dict.get('gamma')
    sgd = input_dict.get('sgd')
    func_type = input_dict.get('func_type')

    w, loss = eval_func(y_tr, x_tr, lambda_, initial_w, max_iters, gamma, sgd)
    _, train_acc = compute_statistics_all(y_tr, x_tr, w, func_type = input_dict['func_type'])
    test_loss, test_acc = compute_statistics_all(y_te, x_te, w, func_type = input_dict['func_type'])
    loss_list.append(test_loss)
    acc_list.append(test_acc)
  return np.mean(loss_list), np.mean(acc_list)

In [7]:
lambdas = np.logspace(-10, 0, 10)
gammas = [0.05,0.1]
max_iters = [1000]
test_func = reg_logistic_regression
# cleans = [0, 1]
cleans = [1]
polys = [1]
degrees = [10,9,8,1,2,3]
normalizes = [1]
# normalizes = [0, 1]
all_polys = [True, False]

In [8]:
train_label_reg = train_label.copy()
train_label_reg[train_label_reg == -1] = 0

In [9]:
train_label_reg

array([1., 0., 0., ..., 1., 0., 0.])

In [10]:
k_fold_num = 5
group_nums = [2,1,0]

In [ ]:
# 3 sigma 
import warnings
warnings.filterwarnings("ignore")
num_iter_ = 0
max_iters = [100]
max_acc = 0
print(train_label_reg[:5])
for lambda_ in lambdas:
  for group_num in group_nums:
    for clean in cleans:
      for poly in polys:
        if poly:
          for gamma in gammas:
            for degree in degrees:
              degree_list = [1] * 3
              degree_list[group_num] = degree
              for normalize_ in normalizes:
                for all_poly in all_polys:
                  group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                                data_utils.process_data(train_data, train_label_reg, clean = clean)
                  group0_x, group1_x, group2_x = \
                          data_utils.group_poly(group0_x, group1_x, group2_x, degree_list, all_poly)
                  cur_group = [group0_x, group1_x, group2_x]
                  cur_labels = [group0_labels, group1_labels, group2_labels]
                  group_x = cur_group[group_num]
                  group_label = cur_labels[group_num]
                  if(normalize_):
                    group_x = data_utils.normalize(group_x)

                  input_dict = {'lambda_':lambda_, \
                          'initial_w':np.zeros(group_x.shape[1]), \
                          'max_iters':100, \
                          'gamma':gamma, \
                          'sgd':False, \
                          'func_type':'logistic'}
                  loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
                  print(f'For this combination scheme in group{group_num}.\iter:{num_iter_} \
                        Clean:{clean}, \
                        Poly:{poly}, \
                        Degree:{degree}, \
                        Normalize: {normalize_}, \
                        all_poly:{all_poly}, \
                        Acc:{acc}, Gamma:{gamma}, lambda:{lambda_}')
                  if(acc > max_acc):
                    print(f'Found a better scheme in group{group_num}.\ iter:{num_iter_} \ Clean:{clean}, \
                        Poly:{poly}, \
                        Degree:{degree}, \
                        Normalize: {normalize_}, \
                        all_poly:{all_poly}, \
                        Acc:{acc}, Gamma:{gamma}, lambda:{lambda_}')
                    max_acc = acc
                  num_iter_ = num_iter_ + 1
        else:
          for gamma in gammas:
            for normalize_ in normalizes:
              group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                            data_utils.process_data(train_data, train_label_reg, clean = clean)
              cur_group = [group0_x, group1_x, group2_x]
              cur_labels = [group0_labels, group1_labels, group2_labels]
              group_x = cur_group[group_num]
              group_label = cur_labels[group_num]
              if(normalize_):
                group_x = data_utils.normalize(group_x)
              input_dict = {'lambda_':lambda_, \
                      'initial_w':np.zeros(group_x.shape[1]), \
                      'max_iters':100, \
                      'gamma':gamma, \
                      'sgd':False, \
                      'func_type':'logistic'}
              try:
                loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
              except:
                print(f'Wrong. \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Normalize: {normalize_}')
                continue
              print(f'For this combination scheme in group{group_num}.\iter:{num_iter_} \
                      Clean:{clean}, \
                      Poly:{poly}, \
                      Degree:{degree}, \
                      Normalize: {normalize_}, \
                      all_poly:{all_poly}, \
                      Acc:{acc}, Gamma:{gamma}, lambda:{lambda_}')
              if(acc > max_acc):
                print(f'Found a better scheme in group{group_num}. \iter:{num_iter_} \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Normalize: {normalize_}, \
                    Acc:{acc}, gamma:{gamma}, lambda:{lambda_}')
                max_acc = acc
              num_iter_ = num_iter_ + 1

[1. 0. 0. 0. 0.]
For this combination scheme in group2.\iter:0                         Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:True,                         Acc:0.6897849462365591, Gamma:0.05, lambda:1e-10
Found a better scheme in group2.\ iter:0 \ Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:True,                         Acc:0.6897849462365591, Gamma:0.05, lambda:1e-10
For this combination scheme in group2.\iter:1                         Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:False,                         Acc:0.6841742486903776, Gamma:0.05, lambda:1e-10
For this combination scheme in group2.\iter:2                         Clean:1,                         Poly:1,           

Found a better scheme in group0.\ iter:60 \ Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:True,                         Acc:0.7481133019717745, Gamma:0.1, lambda:1e-10


In [ ]:
# 3 sigma 
import warnings
warnings.filterwarnings("ignore")
num_iter_ = 0
max_iters = [100]
max_acc = 0
print(train_label_reg[:5])
for lambda_ in [1e-10]:
  for group_num in [0]:
    for clean in [1]:
      for poly in [1]:
        if poly:
          for gamma in gammas:
            for degree in [10,9,8]:
              degree_list = [1] * 3
              degree_list[group_num] = degree
              for normalize_ in [1]:
                for all_poly in [1]:
                  group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                                data_utils.process_data(train_data, train_label_reg, clean = clean)
                  group0_x, group1_x, group2_x = \
                          data_utils.group_poly(group0_x, group1_x, group2_x, degree_list, all_poly)
                  cur_group = [group0_x, group1_x, group2_x]
                  cur_labels = [group0_labels, group1_labels, group2_labels]
                  group_x = cur_group[group_num]
                  group_label = cur_labels[group_num]
                  if(normalize_):
                    group_x = data_utils.normalize(group_x)

                  input_dict = {'lambda_':lambda_, \
                          'initial_w':np.zeros(group_x.shape[1]), \
                          'max_iters':5000, \
                          'gamma':gamma, \
                          'sgd':False, \
                          'func_type':'logistic'}
                  loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
                  print(f'For this combination scheme in group{group_num}.\iter:{num_iter_} \
                        Clean:{clean}, \
                        Poly:{poly}, \
                        Degree:{degree}, \
                        Normalize: {normalize_}, \
                        all_poly:{all_poly}, \
                        Acc:{acc}, Gamma:{gamma}, lambda:{lambda_}')
                  if(acc > max_acc):
                    print(f'Found a better scheme in group{group_num}.\ iter:{num_iter_} \ Clean:{clean}, \
                        Poly:{poly}, \
                        Degree:{degree}, \
                        Normalize: {normalize_}, \
                        all_poly:{all_poly}, \
                        Acc:{acc}, Gamma:{gamma}, lambda:{lambda_}')
                    max_acc = acc
                  num_iter_ = num_iter_ + 1
        else:
          for gamma in gammas:
            for normalize_ in normalizes:
              group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                            data_utils.process_data(train_data, train_label_reg, clean = clean)
              cur_group = [group0_x, group1_x, group2_x]
              cur_labels = [group0_labels, group1_labels, group2_labels]
              group_x = cur_group[group_num]
              group_label = cur_labels[group_num]
              if(normalize_):
                group_x = data_utils.normalize(group_x)
              input_dict = {'lambda_':lambda_, \
                      'initial_w':np.zeros(group_x.shape[1]), \
                      'max_iters':5000, \
                      'gamma':gamma, \
                      'sgd':False, \
                      'func_type':'logistic'}
              try:
                loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
              except:
                print(f'Wrong. \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Normalize: {normalize_}')
                continue
              print(f'For this combination scheme in group{group_num}.\iter:{num_iter_} \
                      Clean:{clean}, \
                      Poly:{poly}, \
                      Degree:{degree}, \
                      Normalize: {normalize_}, \
                      all_poly:{all_poly}, \
                      Acc:{acc}, Gamma:{gamma}, lambda:{lambda_}')
              if(acc > max_acc):
                print(f'Found a better scheme in group{group_num}. \iter:{num_iter_} \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Normalize: {normalize_}, \
                    Acc:{acc}, gamma:{gamma}, lambda:{lambda_}')
                max_acc = acc
              num_iter_ = num_iter_ + 1

[1. 0. 0. 0. 0.]
For this combination scheme in group0.\iter:0                         Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:1,                         Acc:0.8330797717946151, Gamma:0.05, lambda:1e-10
Found a better scheme in group0.\ iter:0 \ Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:1,                         Acc:0.8330797717946151, Gamma:0.05, lambda:1e-10
For this combination scheme in group0.\iter:1                         Clean:1,                         Poly:1,                         Degree:9,                         Normalize: 1,                         all_poly:1,                         Acc:0.8327995195676108, Gamma:0.05, lambda:1e-10
For this combination scheme in group0.\iter:2                         Clean:1,                         Poly:1,                      

In [ ]:
group1 = train_data[np.where(train_data[:, 22] >= 2)]
group1[:, 23]

array([ 67.435,  90.547, 123.01 , ...,  91.581, 155.864, 141.752])

In [11]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import sys
import sys

eps = sys.float_info.epsilon  # a small number for numerical stability


def load_data(path):
    label = np.genfromtxt(path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    all_data = np.genfromtxt(path, delimiter=",", skip_header=1)

    data = all_data[:, 2:]

    labels = np.ones(len(label))
    labels[np.where(label == 'b')] = -1

    return data, labels


def group_data_jetnum(x, labels):
    feature_idx_dict = {0: [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                        1: [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 29],
                        2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25,
                            26, 27, 28, 29]}

    jetnum = 22

    group0 = x[np.where(x[:, jetnum] == 0)]
    group0_x = group0[:, feature_idx_dict[0]]
    group0_labels = labels[np.where(x[:, jetnum] == 0)]

    group1 = x[np.where(x[:, jetnum] == 1)]
    # print(group1)
    group1_x = group1[:, feature_idx_dict[1]]
    group1_labels = labels[np.where(x[:, jetnum] == 1)]

    group2 = x[np.where(x[:, jetnum] >= 2)]
    group2_x = group2[:, feature_idx_dict[2]]
    group2_labels = labels[np.where(x[:, jetnum] >= 2)]

    return group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels


def handle_outlier(x, n=3, how="sigma"):
    '''
    Input:
        x: input data

        n:
        n = 3 for n-sigma
        n usually <= 0.5 when using IQR

        how: "sigma" / "IQR"

    Output:
        sigma:
    '''
    D = x.shape[1]
    method = how
    if method == "sigma":
        for dim in range(D):
            cur_dim_x = x[:, dim]

            cur_mean = np.mean(cur_dim_x)
            cur_std = np.std(cur_dim_x)
            sigma_3 = cur_std * float(n)

            mean_3sigma_up = cur_mean + sigma_3
            mean_3sigma_below = cur_mean - sigma_3

            data_inside_3sigma = [v for v in cur_dim_x if (v >= mean_3sigma_below) and (v <= mean_3sigma_up)]

            no_outlier = len(data_inside_3sigma) == 0
            if not no_outlier:
                cur_dim_x[np.where(cur_dim_x > mean_3sigma_up)] = np.max(data_inside_3sigma)
                cur_dim_x[np.where(cur_dim_x < mean_3sigma_below)] = np.min(data_inside_3sigma)

            x[:, dim] = cur_dim_x
    elif method == "IQR":
        for dim in range(D):
            cur_dim_x = x[:, dim]

            quantile1 = np.quantile(cur_dim_x, 0.25, axis=0)
            quantile2 = np.quantile(cur_dim_x, 0.75, axis=0)
            iqr = quantile2 - quantile1  # compute the Inter-quartile Range (IQR)
            lower_bound = quantile1 - n * iqr
            upper_bound = quantile2 + n * iqr

            mean = np.mean(cur_dim_x)

            data_within_range = [v for v in cur_dim_x if (v >= lower_bound) and (v <= upper_bound)]
            no_outlier = len(data_within_range)
            if not no_outlier:
                cur_dim_x[np.where(cur_dim_x > upper_bound)] = np.max(data_within_range)
                cur_dim_x[np.where(cur_dim_x < lower_bound)] = np.min(data_within_range)

            x[:, dim] = cur_dim_x
    return x


def build_log(x):
    '''log(x) as feature'''
    N = x.shape[0]
    D = x.shape[1]
    x_log = np.zeros((N, 2 * D))
    x_log[:, :D] = x
    x_log[:, D:] = np.log(x)

    return x_log


def build_poly(x, degree):
    '''Augment x with poly basis function'''
    N = x.shape[0]
    D = x.shape[1]
    x_poly = np.zeros((N, D * (degree)))
    for d in range(1, degree + 1):
        x_poly[:, (d - 1) * D:(d) * D] = np.power(x, d)
    return x_poly


def group_poly(group0_x, group1_x, group2_x, degree_list, all_poly=True):
    '''Due to the different dimensionaility of features, perform the '''
    if (all_poly):
        group0_x = build_poly(group0_x, degree_list[0])
        group1_x = build_poly(group1_x, degree_list[1])
        group2_x = build_poly(group2_x, degree_list[2])

    elif (all_poly == False):
        group0_x_poly = build_poly(group0_x[:, :10], degree_list[0])
        group0_x = np.concatenate([group0_x_poly, group0_x[:, 10:]], axis=1)

        group1_x_poly = build_poly(group1_x[:, :10], degree_list[1])
        group1_x = np.concatenate([group1_x_poly, group1_x[:, 10:]], axis=1)

        group2_x_poly = build_poly(group2_x[:, :13], degree_list[2])
        group2_x = np.concatenate([group2_x_poly, group2_x[:, 13:]], axis=1)

    return group0_x, group1_x, group2_x


def standardize(x):
    mean_x = np.mean(x, axis=0, keepdims=True)
    x = x - mean_x
    std_x = np.std(x, axis=0, keepdims=True)
    x = x / (std_x + eps)
    return x


def normalize(x):
    D = x.shape[1]
    for d in range(D):
        if (np.all(x[:, d] == 1)):
            '''When build poly there can be all 1 column.'''
            pass
        else:
            x[:, d] = (x[:, d] - np.min(x[:, d])) / (np.max(x[:, d]) - np.min(x[:, d]))
    return x


def process_data1(x, labels, train=True, clean=True):
    if (train):
        broken_feature = x[:, 0]
        broken_feature_no_nan = broken_feature[np.where(broken_feature != -999.0)]
        fill_na_value = np.median(broken_feature_no_nan)
        broken_feature[np.where(broken_feature == -999.0)] = fill_na_value
        x[:, 0] = broken_feature
    group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = group_data_jetnum(x, labels)
    if (clean):
        group0_x = handle_outlier(group0_x, n=1)
        group1_x = handle_outlier(group1_x, n=1)
        group2_x = handle_outlier(group2_x, n=1)
    return group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels


def k_fold(data_num, fold, seed=114514):
    '''Return a array containing {fold} elements of indices'''
    np.random.seed(seed)
    test_num = data_num // fold
    idx_list = np.random.permutation(data_num)
    fold_part_idx = np.array([idx_list[x * test_num: (x + 1) * test_num] for x in range(fold)])
    return fold_part_idx





In [ ]:
# 1sigma  lambda - 1e-10
import warnings
warnings.filterwarnings("ignore")
num_iter_ = 0
max_acc = 0
group_nums = [2,1,0]
print(train_label_reg[:5])
for lambda_ in [1e-10]:
  for group_num in [0]:
    for clean in [1]:
      for poly in [1]:
        if poly:
          for gamma in gammas:
            for degree in [10,9,8]:
              degree_list = [1] * 3
              degree_list[group_num] = degree
              for normalize_ in [1]:
                for all_poly in [1]:
                  group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                                process_data1(train_data, train_label_reg, clean = clean)
                  group0_x, group1_x, group2_x = \
                          group_poly(group0_x, group1_x, group2_x, degree_list, all_poly)
                  cur_group = [group0_x, group1_x, group2_x]
                  cur_labels = [group0_labels, group1_labels, group2_labels]
                  group_x = cur_group[group_num]
                  group_label = cur_labels[group_num]
                  if(normalize_):
                    group_x = data_utils.normalize(group_x)

                  input_dict = {'lambda_':1e-10, \
                          'initial_w':np.zeros(group_x.shape[1]), \
                          'max_iters':5000, \
                          'gamma':gamma, \
                          'sgd':False, \
                          'func_type':'logistic'}
                  loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
                  print(f'For this combination scheme in group{group_num}.\iter:{num_iter_} \
                        Clean:{clean}, \
                        Poly:{poly}, \
                        Degree:{degree}, \
                        Normalize: {normalize_}, \
                        all_poly:{all_poly}, \
                        Acc:{acc}, Gamma:{gamma}')
                  if(acc > max_acc):
                    print(f'Found a better scheme in group{group_num}.\ iter:{num_iter_} \ Clean:{clean}, \
                        Poly:{poly}, \
                        Degree:{degree}, \
                        Normalize: {normalize_}, \
                        all_poly:{all_poly}, \
                        Acc:{acc}, Gamma:{gamma}')
                    max_acc = acc
                  num_iter_ = num_iter_ + 1
        else:
          for gamma in gammas:
            for normalize_ in [1]:
              group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                            process_data1(train_data, train_label_reg, clean = clean)
              cur_group = [group0_x, group1_x, group2_x]
              cur_labels = [group0_labels, group1_labels, group2_labels]
              group_x = cur_group[group_num]
              group_label = cur_labels[group_num]
              if(normalize_):
                group_x = data_utils.normalize(group_x)
              input_dict = {'lambda_':1e-10, \
                      'initial_w':np.zeros(group_x.shape[1]), \
                      'max_iters':5000, \
                      'gamma':gamma, \
                      'sgd':False, \
                      'func_type':'logistic'}
              try:
                loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
              except:
                print(f'Wrong. \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Normalize: {normalize_}')
                continue
              print(f'For this combination scheme in group{group_num}.\iter:{num_iter_} \
                      Clean:{clean}, \
                      Poly:{poly}, \
                      Degree:{degree}, \
                      Normalize: {normalize_}, \
                      all_poly:{all_poly}, \
                      Acc:{acc}, Gamma:{gamma}')
              if(acc > max_acc):
                print(f'Found a better scheme in group{group_num}. \iter:{num_iter_} \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Normalize: {normalize_}, \
                    Acc:{acc}, gamma:{gamma}')
                max_acc = acc
              num_iter_ = num_iter_ + 1

[1. 0. 0. 0. 0.]
For this combination scheme in group0.\iter:0                         Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:1,                         Acc:0.8429786808127314, Gamma:0.05
Found a better scheme in group0.\ iter:0 \ Clean:1,                         Poly:1,                         Degree:10,                         Normalize: 1,                         all_poly:1,                         Acc:0.8429786808127314, Gamma:0.05
For this combination scheme in group0.\iter:1                         Clean:1,                         Poly:1,                         Degree:9,                         Normalize: 1,                         all_poly:1,                         Acc:0.8424381943749374, Gamma:0.05
For this combination scheme in group0.\iter:2                         Clean:1,                         Poly:1,                         Degree:8,                         Norma

In [12]:
def add_bias(x):
    N, D = x.shape
    bias_x = np.ones((N, D+1))
    bias_x[:, :D] = x
    return bias_x

In [16]:
# 1sigma + bias
import warnings
warnings.filterwarnings("ignore")
num_iter_ = 0
max_acc = 0
degrees = [10,9,8]
ploys=[1]
cleans=[1]
bias = True
group_num=0
print(train_label_reg[:5])
for clean in cleans:
  for poly in polys:
    if poly:
      for gamma in [0.1]:
        for degree in [10]:
          degree_list = [1] * 3
          degree_list[group_num] = degree
          for normalize_ in [1]:
            for all_poly in all_polys:
              group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                            process_data1(train_data, train_label_reg, clean = clean)
              group0_x, group1_x, group2_x = \
                      group_poly(group0_x, group1_x, group2_x, degree_list, all_poly)
              cur_group = [group0_x, group1_x, group2_x]
              cur_labels = [group0_labels, group1_labels, group2_labels]
              group_x = cur_group[group_num]
              group_label = cur_labels[group_num]
              if(normalize_):
                group_x = data_utils.normalize(group_x)
              if(bias):
                group_x = add_bias(group_x)
              input_dict = {'lambda_':0, \
                      'initial_w':np.zeros(group_x.shape[1]), \
                      'max_iters':8000, \
                      'gamma':gamma, \
                      'sgd':False, \
                      'func_type':'logistic'}
              loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
              print(f'For this combination scheme.\iter:{num_iter_} \
                    Clean:{clean}, \
                    Poly:{poly}, \
                    Degree:{degree}, \
                    Normalize: {normalize_}, \
                    all_poly:{all_poly}, \
                    Acc:{acc}, Gamma:{gamma}')
              if(acc > max_acc):
                print(f'Found a better scheme.\ iter:{num_iter_} \ Clean:{clean}, \
                    Poly:{poly}, \
                    Degree:{degree}, \
                    Normalize: {normalize_}, \
                    all_poly:{all_poly}, \
                    Acc:{acc}, Gamma:{gamma}')
                max_acc = acc
              num_iter_ = num_iter_ + 1
    else:
      for gamma in [0.1]:
        for normalize_ in [1]:
          group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                        process_data1(train_data, train_label_reg, clean = clean)
          cur_group = [group0_x, group1_x, group2_x]
          cur_labels = [group0_labels, group1_labels, group2_labels]
          group_x = cur_group[group_num]
          group_label = cur_labels[group_num]
          if(normalize_):
            group_x = data_utils.normalize(group_x)
          if(bias):
            group_x = add_bias(group_x)
          input_dict = {'lambda_':1e-10, \
                  'initial_w':np.zeros(group_x.shape[1]), \
                  'max_iters':10000, \
                  'gamma':gamma, \
                  'sgd':False, \
                  'func_type':'logistic'}
          try:
            loss, acc = cross_validation(group_x, group_label, k_fold_num, test_func, input_dict)
          except:
            print(f'Wrong. \
                Clean:{clean}, \
                Poly:{poly}, \
                Normalize: {normalize_}')
            continue
          print(f'For this combination scheme.\iter:{num_iter_} \
                  Clean:{clean}, \
                  Poly:{poly}, \
                  Degree:{degree}, \
                  Normalize: {normalize_}, \
                  all_poly:{all_poly}, \
                  Acc:{acc}, Gamma:{gamma}')
          if(acc > max_acc):
            print(f'Found a better scheme. \iter:{num_iter_} \
                Clean:{clean}, \
                Poly:{poly}, \
                Normalize: {normalize_}, \
                Acc:{acc}, gamma:{gamma}')
            max_acc = acc
          num_iter_ = num_iter_ + 1

[1. 0. 0. 0. 0.]
For this combination scheme.\iter:0                     Clean:1,                     Poly:1,                     Degree:10,                     Normalize: 1,                     all_poly:True,                     Acc:0.8446902211990791, Gamma:0.1
Found a better scheme.\ iter:0 \ Clean:1,                     Poly:1,                     Degree:10,                     Normalize: 1,                     all_poly:True,                     Acc:0.8446902211990791, Gamma:0.1
For this combination scheme.\iter:1                     Clean:1,                     Poly:1,                     Degree:10,                     Normalize: 1,                     all_poly:False,                     Acc:0.8434190771694524, Gamma:0.1


In [19]:

def load_csv_data(data_path, sub_sample=False):
    """Loads data and returns y (class labels), tX (features) and ids (event ids)"""
    y = np.genfromtxt(data_path, delimiter=",", skip_header=1, dtype=str, usecols=1)
    x = np.genfromtxt(data_path, delimiter=",", skip_header=1)
    ids = x[:, 0].astype(np.int)
    input_data = x[:, 2:]

    # convert class labels from strings to binary (-1,1)
    yb = np.ones(len(y))
    yb[np.where(y == "b")] = -1

    # sub-sample
    if sub_sample:
        yb = yb[::50]
        input_data = input_data[::50]
        ids = ids[::50]

    return yb, input_data, ids
degree_list = [1] * 3
degree_list[0] = 10
group0_x, group0_labels, group1_x, group1_labels, group2_x, group2_labels = \
                            process_data1(train_data, train_label_reg, clean = True)
cur_group = [group0_x, group1_x, group2_x]
cur_labels = [group0_labels, group1_labels, group2_labels]
group_x = cur_group[0]
group_x = data_utils.normalize(group_x)
group_x = add_bias(group_x)
group_label = cur_labels[0]
w, loss = reg_logistic_regression(group_label, group_x, lambda_ = 1e-10, initial_w=np.zeros(group_x.shape[1]),max_iters = 50000, gamma = 0.1)

feature_idx_dict = {0: [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21],
                        1: [0, 1, 2, 3, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 29],
                        2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25,
                            26, 27, 28, 29]}
test_label, test_data, ids = load_csv_data(test_set_root, sub_sample=False)
test_data = test_data[:, feature_idx_dict[0]]
test_data = add_bias(test_data)
model_output = sigmoid(np.dot(test_data, w))
# loss = -np.dot(y, np.log(model_output + eps)) - np.dot((1 - y).T, np.log(1 - model_output + eps))
model_output[np.where(model_output > 0.5)] = 1
model_output[np.where(model_output <= 0.5)] = -1
model_output


array([ 1.,  1., -1., ...,  1.,  1., -1.])